# 🎯 Day 2 실습 8: RAG 튜닝 & 커스터마이징 - 나만의 RAG 만들기

## 🎯 학습 목표
- **파라미터 튜닝**: Chunk Size, Top-K, Threshold 직접 조정해보기
- **최적 설정 찾기**: 질문 타입별 최적 파라미터 조합 발견
- **커스텀 데이터셋**: 나만의 평가 데이터셋 만들기
- **빠른 프로토타입**: Gradio로 3줄 UI 만들기

## 💡 핵심 아이디어

```
지금까지 배운 기법들:
  01번: Naive RAG
  03번: Multi-Query
  04번: Metadata Filtering
  05번: Hybrid + Reranking
  06번: Corrective RAG
  07번: RAG Evaluation

하지만... 언제 어떤 설정을 써야 할까?

이번 실습에서는:
  ✅ 직접 파라미터를 조정해보고
  ✅ 결과를 관찰하고
  ✅ 나만의 최적 설정을 찾아봅니다!
```

## 📋 실습 구성
1. **Part 1**: 파라미터 튜닝 실험실 (Chunk Size, Top-K, Threshold)
2. **Part 2**: 나만의 평가 데이터셋 만들기
3. **Part 3**: Gradio UI로 빠른 프로토타입
4. **Part 4**: 최종 정리 & Day 3 Preview

---

> 💡 **이번 실습의 특별함**
> - 기존: 코드 실행만 → 이번: **직접 수정하고 실험**
> - 기존: 정해진 답 → 이번: **나만의 최적 설정 발견**
> - 기존: 보기만 → 이번: **만들어보기**

In [ ]:
# 라이브러리 설치 및 import
!pip install -q langchain-community faiss-cpu sentence-transformers pandas numpy scikit-learn

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity

print("✅ 라이브러리 준비 완료!")

In [ ]:
# 데이터 및 평가 함수 정의

# 13개 레시피 문서 (핵심 5 + Confuser 5 + 노이즈 3)
recipe_docs_raw = [
    # 핵심 5개 (정통 레시피)
    {"recipe_name": "김치찌개", "content": "김치찌개 만드는 법: 돼지고기를 참기름에 볶다가 김치를 넣고 볶는다. 물을 붓고 두부, 대파, 고춧가루를 넣어 끓인다. 간은 국간장과 새우젓으로 맞춘다. 매콤하고 칼칼한 맛이 일품이다."},
    {"recipe_name": "된장찌개", "content": "된장찌개 만드는 법: 멸치와 다시마로 육수를 낸다. 된장을 풀고 감자, 호박, 두부를 넣는다. 청양고추와 대파를 넣어 마무리한다. 구수하고 얼큰한 맛이 특징이다."},
    {"recipe_name": "제육볶음", "content": "제육볶음 만드는 법: 돼지고기를 고춧가루, 간장, 설탕, 다진 마늘로 양념한다. 양파와 함께 센 불에서 볶는다. 대파를 넣고 마무리한다. 매콤달콤한 맛이 밥도둑이다."},
    {"recipe_name": "불고기", "content": "불고기 만드는 법: 소고기를 간장, 설탕, 배즙, 참기름, 다진 마늘로 재운다. 양파, 버섯과 함께 볶는다. 당면을 추가하면 더 맛있다. 달콤하고 부드러운 맛이 특징이다."},
    {"recipe_name": "비빔밥", "content": "비빔밥 만드는 법: 밥 위에 나물(시금치, 고사리, 도라지), 고기, 계란 프라이를 올린다. 고추장을 넣고 참기름을 두른다. 잘 비벼 먹는다. 영양 만점 한 그릇 식사다."},
    
    # Confuser 5개 (비슷하지만 다른 레시피 - TV 프로그램 변형)
    {"recipe_name": "백종원 참치김치찌개", "content": "백종원 참치김치찌개: 참치캔을 기름째 넣는 게 포인트다. 김치와 참치를 볶다가 물을 붓는다. 두부와 대파를 넣고 끓인다. 참치의 감칠맛이 일품이다. 백종원의 골목식당에서 소개된 레시피다."},
    {"recipe_name": "흑백요리사 이모카세 두부찌개", "content": "흑백요리사 이모카세 두부찌개: 된장과 고추장을 섞어 쓴다. 두부를 큼직하게 썰어 넣는다. 청양고추를 많이 넣어 매운맛을 낸다. 이모카세 셰프의 비법이 담긴 레시피다."},
    {"recipe_name": "백종원 제육볶음", "content": "백종원 제육볶음: 고추장 대신 고춧가루를 많이 쓴다. 양파를 많이 넣어 단맛을 낸다. 물을 약간 넣어 자작하게 끓인다. 백종원의 요리비책에서 소개된 황금 레시피다."},
    {"recipe_name": "김치볶음밥", "content": "김치볶음밥 만드는 법: 김치를 잘게 썰어 참기름에 볶는다. 밥을 넣고 볶다가 김치국물을 추가한다. 계란 프라이를 올려 마무리한다. 간단하지만 맛있는 한 끼 식사다."},
    {"recipe_name": "순두부찌개", "content": "순두부찌개 만드는 법: 멸치 육수에 고춧가루를 풀어 국물을 낸다. 순두부를 넣고 조개, 새우를 추가한다. 계란을 풀어 넣는다. 얼큰하고 부드러운 맛이 일품이다."},
    
    # 노이즈 3개 (관련 없는 정보)
    {"recipe_name": "한식 조리 용어", "content": "한식 조리 용어 정리: 볶음(炒)은 기름에 재료를 익히는 것. 조림(煮)은 양념장에 재료를 넣고 졸이는 것. 찜(蒸)은 수증기로 익히는 것. 무침은 양념에 버무리는 것."},
    {"recipe_name": "음식 보관법", "content": "음식 보관법: 김치는 냉장 보관한다. 두부는 물에 담가 냉장 보관한다. 고기는 냉동 보관이 좋다. 나물은 밀폐 용기에 담아 냉장 보관한다."},
    {"recipe_name": "칼 사용법", "content": "칼 사용법: 채칼은 채를 썰 때, 식도는 생선을 손질할 때, 중식도는 고기를 자를 때 사용한다. 도마는 용도별로 구분해서 쓴다."},
]

# 임베딩 모델
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Context Precision (코사인 유사도 기반)
def calculate_context_precision(query, retrieved_docs, embeddings_model, threshold=0.3):
    """
    Precision = (검색된 문서 중 관련 있는 것) / (검색된 전체)
    
    관련성 판단: query와 doc의 코사인 유사도 >= threshold
    """
    from sklearn.metrics.pairwise import cosine_similarity
    
    query_embedding = embeddings_model.embed_query(query)
    relevant_count = 0
    
    for doc in retrieved_docs:
        doc_embedding = embeddings_model.embed_query(doc.page_content)
        similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
        
        if similarity >= threshold:
            relevant_count += 1
    
    precision = relevant_count / len(retrieved_docs) if retrieved_docs else 0.0
    return precision, relevant_count

# Context Recall (키워드 매칭 기반)
def calculate_context_recall(required_recipes, retrieved_docs):
    """
    Recall = (찾은 정답) / (실제 정답 전체)
    
    정답 판단: metadata의 recipe_name이 required_recipes에 있는지
    """
    retrieved_recipes = [doc.metadata.get('recipe_name', '') for doc in retrieved_docs]
    found_count = 0
    for required in required_recipes:
        if required in retrieved_recipes:
            found_count += 1
    recall = found_count / len(required_recipes) if required_recipes else 0.0
    return recall, found_count

print("✅ 데이터 및 평가 함수 준비 완료!")
print(f"   레시피 수: {len(recipe_docs_raw)}개")
print(f"   임베딩 모델: BAAI/bge-m3")
print()
print("💡 평가 방식:")
print("   - Precision: query와 doc의 코사인 유사도 >= threshold")
print("   - Recall: required_recipes가 검색 결과에 있는지 (metadata 기반)")

## 📚 실습 데이터: 레시피 코퍼스

### 데이터 구성

**왜 이 데이터를 사용하나요?**
- 파라미터 튜닝 효과를 **실제로 체험**하려면 적절한 데이터가 필요합니다
- 너무 짧으면 → chunk_size 변경해도 차이 없음
- 너무 간단하면 → 항상 성공해서 튜닝 의미 없음

**우리가 준비한 데이터:**

| 타입 | 개수 | 평균 길이 | 목적 |
|------|------|-----------|------|
| **핵심 레시피** | 5개 | ~900자 | 긴 조리법 포함 (파라미터 튜닝 효과 극대화) |
| **Confuser 레시피** | 5개 | ~300자 | 비슷한 요리 (김치찌개 vs 참치김치찌개) |
| **노이즈 문서** | 3개 | ~200자 | 일반 요리 정보 |
| **합계** | **13개** | **~700자** | 충분한 실험 환경 |

### 핵심 레시피 예시 (김치찌개)

```
【재료】돼지고기 200g, 묵은김치 300g, 두부 1/2모, 대파 1대...

【조리법】
1단계: 돼지고기를 먹기 좋은 크기로 자른다
2단계: 김치는 한입 크기로 자른다
...
```

**💡 왜 이렇게 구성했나요?**
1. **길이 다양성** - 파라미터 튜닝 효과를 확인하려면 충분한 길이 필요
2. **혼란 요소** - 비슷한 레시피로 Precision/Recall 차이 확인
3. **실전성** - 실제 RAG 시스템도 이런 환경에서 작동

---

### 테스트 질문들

실습에서 사용할 질문 3가지:

1. **간단한 질문** (1개 레시피) - "김치찌개 만드는 법 알려줘"
2. **복잡한 질문** (2개 레시피) - "찌개 종류 중에 두부 들어가고 매운 거 뭐 있어?"
3. **애매한 질문** (1개 레시피) - "고기 들어가는 매운 요리 추천해줘"

→ chunk_size, top_k를 조절하면서 어떻게 변하는지 관찰하세요!

### 📊 결과 해석 가이드

**실험 전에 꼭 읽어보세요!**

#### ⚠️ Precision과 Recall의 계산 방식

**이 노트북의 하이브리드 방식:**

| 지표 | 계산 방법 | 예시 |
|------|-----------|------|
| **Precision** | (검색된 문서 중 관련 있는 것) / (검색된 전체) | 3개 검색, 2개 관련 → 2/3 = 0.67 |
| **Recall** | (찾은 정답) / (실제 정답 전체) | 2개 필요, 1개 찾음 → 1/2 = 0.50 |

**현재 구현:**

```python
# Precision: 코사인 유사도 기반 (유연한 관련성 판단)
similarity = cosine_similarity(query_embedding, doc_embedding)
if similarity >= 0.3:  # threshold
    relevant_count += 1

# Recall: 키워드 매칭 기반 (명확한 정답 판단)
if retrieved_recipe in required_recipes:
    found_count += 1
```

---

#### 🔍 왜 하이브리드 방식인가?

| 지표 | 방식 | 이유 |
|------|------|------|
| **Precision** | 코사인 유사도 (threshold=0.3) | ✅ 의미적 유사도 반영<br>✅ 실전 RAG와 일치<br>✅ 유연한 관련성 판단 |
| **Recall** | 키워드 매칭 (metadata) | ✅ 정답/오답 명확<br>✅ "필요한 문서를 찾았는지" 명확하게 판단 |

**💡 실전 RAG에서 가장 많이 쓰는 방식입니다!**
- Precision: "검색 품질" → 코사인 유사도로 평가
- Recall: "정답 커버리지" → 정답 문서를 찾았는지 평가

---

#### ❓ 왜 Recall은 exact match인가?

**Q: Recall도 코사인 유사도로 하면 안 되나요?**

**A: 안 됩니다! 이유:**

**1. 정답을 못 찾아도 Recall이 높게 나올 수 있음**

```
질문: "찌개 종류 중에 두부 들어가고 매운 거 뭐 있어?"
정답 (필요한 레시피): ["김치찌개", "된장찌개"]

검색 결과:
1. 순두부찌개 (유사도 0.8) ✅ 코사인 유사도 높음
2. 흑백요리사 두부찌개 (유사도 0.75) ✅ 코사인 유사도 높음
3. 백종원 참치김치찌개 (유사도 0.7) ✅ 코사인 유사도 높음

❌ 만약 Recall도 코사인 유사도 쓰면?
→ Recall = 3/2 = ... 계산이 애매함
→ 또는 threshold 넘는 것 카운트 → 3개 모두 높음 → Recall 높게 나옴

✅ 하지만 실제로는?
→ 김치찌개, 된장찌개를 하나도 못 찾았음!
→ Recall은 0/2 = 0.0이어야 정확함
```

**2. Recall의 목적: "내가 원하는 것을 찾았나?"**

- Precision: "이것들이 관련 있나?" (유연하게 판단 OK)
- **Recall: "내가 원하는 게 여기 있나?" (엄격하게 판단 필요)**

**3. 실전 예시**

```
실무 상황: 법률 RAG 시스템
질문: "2023년 개정된 민법 제750조 내용"
필요한 문서: ["민법 제750조 (2023 개정판)"]

검색 결과:
1. 민법 제749조 (유사도 0.9) ← 관련은 있지만 정답 아님
2. 민법 제750조 (2020 개정판) (유사도 0.85) ← 관련은 있지만 정답 아님
3. 민법 제751조 (유사도 0.8) ← 관련은 있지만 정답 아님

코사인 유사도로 Recall 계산하면?
→ 다 높게 나옴 → Recall 높음
→ ❌ 하지만 정확한 2023년 개정판을 못 찾음!

exact match로 Recall 계산하면?
→ Recall = 0/1 = 0.0
→ ✅ "원하는 문서를 못 찾았다"를 정확히 반영
```

**결론:**
- Precision은 유연하게 (코사인 유사도) → 검색 품질 평가
- Recall은 엄격하게 (exact match) → 정답 커버리지 평가
- 이 조합이 가장 정확한 RAG 평가!

---

#### 💡 실제 예시

**질문:** "찌개 종류 중에 두부 들어가고 매운 거 뭐 있어?"

**정답 (required_recipes):** ["김치찌개", "된장찌개"] (2개)

**검색 결과 (k=3):**
1. 순두부찌개 (유사도: 0.42) ✅ 관련 있음 (threshold=0.3 초과)
2. 흑백요리사 두부찌개 (유사도: 0.38) ✅ 관련 있음
3. 김치찌개 (유사도: 0.51) ✅ 관련 있음, ✅ 정답!

**계산:**

```
Precision = (관련 있는 문서) / (검색된 전체)
         = 3 / 3 = 1.00
         (순두부찌개 0.42, 두부찌개 0.38, 김치찌개 0.51 모두 >= 0.3)

Recall = (찾은 정답) / (실제 정답)
       = 1 / 2 = 0.50
       (김치찌개 O, 된장찌개 X)
```

**해석:**
- Precision 1.00 = 검색된 3개 문서 모두 query와 관련 있음 (검색 품질 좋음)
- Recall 0.50 = 필요한 레시피 2개 중 1개만 찾음 (된장찌개 못 찾음)

---

#### 🤔 왜 이렇게 되는가?

**Precision 높음:**
- 순두부찌개, 흑백요리사 두부찌개도 "찌개 + 두부"라는 측면에서 관련 있음
- → 코사인 유사도로 의미적 관련성을 포착!

**Recall 낮음:**
- 된장찌개를 못 찾음
- → chunk_size가 너무 작거나, top_k가 부족함

---

#### ✅ 파라미터 튜닝 가이드

| 상황 | Precision | Recall | 해결책 |
|------|-----------|--------|--------|
| chunk 너무 작음 | 낮을 수 있음 | ⚠️ 낮음 | chunk ↑ (300~500) |
| k 너무 작음 | 높을 수 있음 | ⚠️ 낮음 | k ↑ (3~5) |
| k 너무 큼 | ⚠️ 낮음 | 높을 수 있음 | k ↓ |
| threshold 너무 높음 (0.7) | ⚠️ 낮음 | 변화 없음 | threshold ↓ (0.3) |
| threshold 너무 낮음 (0.1) | 높음 (노이즈 포함) | 변화 없음 | threshold ↑ (0.3) |
| 균형 잡힘 | ✅ 0.6+ | ✅ 0.8+ | 유지! |

#### 💡 실험 팁

1. **한 번에 하나씩** - chunk만 바꾸고 결과 확인 → k 바꾸기 → threshold 바꾸기
2. **완벽한 1.0이 목표 아님** - Precision ≥ 0.6, Recall ≥ 0.8 이면 성공!
3. **Precision vs Recall 트레이드오프** - k를 늘리면 Recall은 높아지지만 Precision은 낮아질 수 있음
4. **threshold 조절** - Precision이 너무 낮으면 threshold를 높여보세요 (0.3 → 0.5)

---

### 다음: 실제로 실험해보세요!

### 🧪 실습 1: Chunk Size 실험

**과제: `chunk_size`를 바꿔보세요!**

**현재 설정은 의도적으로 나쁩니다!**
- 초기값: chunk_size=**50** (극단적으로 작음 😱)
- 문제: 문맥이 극도로 부족 → 일부 질문에서 실패

**개선 방법:**
- 50 (현재) → 최악! 문맥 거의 없음 ❌
- 100 (매우 작음) → 문맥 부족
- 300 (적당) → 좋음! ✅
- 500 (보통) → 균형 ✅
- 1000 (큼) → 괜찮음
- 2000 (매우 큼) → 노이즈 많음

**중요한 이해:**
- **간단한 질문**에서는 chunk=50도 성공할 수 있습니다
- 하지만 **복잡한 질문**이나 **애매한 질문**에서는 실패합니다
- 목표: **안정적으로** 작동하는 설정 찾기!

**관찰 포인트:**
- chunk_size를 늘리면 어떻게 변하나요?
- 어떤 크기가 가장 균형잡혔나요?
- chunk 개수가 어떻게 변하나요?

In [ ]:
# ============================================
# 여기를 수정하세요! 👇
# ============================================
chunk_size = 50   # ← 이 숫자를 바꿔보세요! (50, 100, 300, 500, 1000)
                  # ⚠️ 현재 50은 의도적으로 나쁜 설정입니다!
                  # 💡 300~500으로 바꿔보세요!

# ============================================

# Text Splitter 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=20,
    length_function=len
)

# 문서 분할
documents = []
for doc_data in recipe_docs_raw:
    chunks = text_splitter.split_text(doc_data["content"])
    for chunk in chunks:
        documents.append(Document(
            page_content=chunk,
            metadata={
                "recipe_name": doc_data["recipe_name"]
            }
        ))

# 벡터스토어 생성
vectorstore = FAISS.from_documents(documents, embeddings)

print(f"✅ Chunk Size = {chunk_size}자로 설정!")
print(f"   전체 레시피: {len(recipe_docs_raw)}개")
print(f"   분할 후 Chunk: {len(documents)}개")
print(f"   평균 Chunk 크기: {np.mean([len(d.page_content) for d in documents]):.0f}자")
print()

if chunk_size >= 1500:
    print("⚠️ 경고: chunk_size가 너무 큽니다!")
    print("   → 긴 레시피가 1개 chunk로 → 노이즈 많음")
    print("   💡 300~500으로 줄여보세요!")
elif chunk_size < 100:
    print("⚠️ 경고: chunk_size가 너무 작습니다!")
    print("   → 문맥이 극도로 부족 → 검색 정확도 매우 낮음")
    print("   💡 300~500으로 늘려보세요!")
elif chunk_size < 200:
    print("⚠️ 경고: chunk_size가 작습니다!")
    print("   → 문맥 부족 → 검색 정확도 낮음")
    print("   💡 300~500으로 늘려보세요!")
else:
    print("✅ 적절한 chunk_size입니다!")

In [ ]:
# 테스트 질문 (복잡한 질문 - 여러 레시피 관련!)
test_query = "찌개 종류 중에 두부 들어가고 매운 거 뭐 있어?"
required_recipes = ["김치찌개", "된장찌개"]

# 검색
results = vectorstore.similarity_search(test_query, k=3)

# 평가
precision, rel_count = calculate_context_precision(test_query, results, embeddings, threshold=0.3)
recall, found_count = calculate_context_recall(required_recipes, results)

# 출력
print(f"🔍 질문: '{test_query}'")
print(f"   정답: {required_recipes}")
print()
print(f"📊 결과 (Chunk={chunk_size}, k=3):")
print(f"   Precision: {precision:.2f} ({rel_count}/3 문서가 관련 있음)")
print(f"   Recall: {recall:.2f} ({found_count}/{len(required_recipes)} 필요 레시피 찾음)")
print()

# 검색된 레시피 이름만 출력
print("📝 검색된 레시피:")
for i, doc in enumerate(results, 1):
    recipe_name = doc.metadata.get('recipe_name', 'Unknown')
    print(f"   {i}. {recipe_name}")
print()

# 간단 해석
if recall < 0.5:
    print("⚠️ Recall 낮음: chunk_size를 늘려보세요!")
elif recall >= 0.8 and precision >= 0.6:
    print("✅ 좋은 결과!")
else:
    print("💡 파라미터 조정으로 개선 가능")

In [ ]:
# 테스트 질문 (완전 간접적 - 키워드 완전 제거!)
test_query = "고기 들어가는 매운 요리 추천해줘"
required_recipes = ["김치찌개"]

# 검색
results = vectorstore.similarity_search(test_query, k=3)

# 평가
precision, rel_count = calculate_context_precision(test_query, results, embeddings, threshold=0.3)
recall, found_count = calculate_context_recall(required_recipes, results)

# 출력
print(f"🔍 질문: '{test_query}'")
print(f"   정답: {required_recipes}")
print()
print(f"📊 결과 (Chunk={chunk_size}, k=3):")
print(f"   Precision: {precision:.2f} ({rel_count}/3 문서가 관련 있음)")
print(f"   Recall: {recall:.2f} ({found_count}/{len(required_recipes)} 필요 레시피 찾음)")
print()

# 검색된 레시피 이름만 출력
print("📝 검색된 레시피:")
for i, doc in enumerate(results, 1):
    recipe_name = doc.metadata.get('recipe_name', 'Unknown')
    print(f"   {i}. {recipe_name}")
print()

if recall == 0:
    print("⚠️ 정답을 못 찾음: 간접적 질문은 chunk_size↑ 또는 k↑ 필요")
elif recall >= 1.0 and precision >= 0.6:
    print("✅ 간접 질문도 성공!")

In [ ]:
# ============================================
# 여기를 수정하세요! 👇
# ============================================
top_k = 1  # ← 이 숫자를 바꿔보세요! (1, 3, 5, 10)
           # ⚠️ 현재 1은 의도적으로 나쁜 설정입니다!
           # 💡 3~5로 바꿔보세요!
# ============================================

# 테스트 질문
test_query = "고기 들어가는 매콤한 볶음요리 추천해줘"
required_recipes = ["제육볶음", "백종원 제육볶음"]

# 검색
results = vectorstore.similarity_search(test_query, k=top_k)

# 평가
precision, rel_count = calculate_context_precision(test_query, results, embeddings, threshold=0.3)
recall, found_count = calculate_context_recall(required_recipes, results)

# 출력
print(f"🔍 질문: '{test_query}'")
print(f"   정답: {required_recipes}")
print()
print(f"📊 결과 (k={top_k}):")
print(f"   Precision: {precision:.2f} ({rel_count}/{top_k} 문서가 관련 있음)")
print(f"   Recall: {recall:.2f} ({found_count}/{len(required_recipes)} 필요 레시피 찾음)")
print()

print("📝 검색된 레시피:")
for i, doc in enumerate(results, 1):
    recipe_name = doc.metadata.get('recipe_name', 'Unknown')
    print(f"   {i}. {recipe_name}")
print()

if top_k < 3:
    print("⚠️ k가 너무 작음: Recall이 낮을 수 있습니다")
elif recall >= 0.8:
    print("✅ 좋은 Recall!")

In [ ]:
# 테스트 질문 (간단한 질문)
test_query = "김치찌개 만드는 법 알려줘"
required_recipes = ["김치찌개"]

# 검색
results = vectorstore.similarity_search(test_query, k=top_k)

# 평가
precision, rel_count = calculate_context_precision(test_query, results, embeddings, threshold=0.3)
recall, found_count = calculate_context_recall(required_recipes, results)

# 출력
print(f"🔍 질문: '{test_query}'")
print(f"   정답: {required_recipes}")
print()
print(f"📊 결과 (k={top_k}):")
print(f"   Precision: {precision:.2f} ({rel_count}/{top_k} 문서가 관련 있음)")
print(f"   Recall: {recall:.2f} ({found_count}/{len(required_recipes)} 필요 레시피 찾음)")
print()

if recall >= 1.0:
    print("✅ 간단한 질문은 성공!")

In [ ]:
# ============================================
# 여기를 수정하세요! 👇
# ============================================
threshold = 0.7  # ← 이 숫자를 바꿔보세요! (0.1, 0.3, 0.5, 0.7)
                 # ⚠️ 현재 0.7은 의도적으로 높은 설정입니다!
                 # 💡 0.3~0.5로 바꿔보세요!
# ============================================

# 테스트 질문
test_query = "간단하게 만들 수 있는 밥 요리"
required_recipes = ["김치볶음밥"]

# 검색
results = vectorstore.similarity_search(test_query, k=3)

# 평가
precision, rel_count = calculate_context_precision(test_query, results, embeddings, threshold=threshold)
recall, found_count = calculate_context_recall(required_recipes, results)

# 출력
print(f"🔍 질문: '{test_query}'")
print(f"   정답: {required_recipes}")
print()
print(f"📊 결과 (threshold={threshold}):")
print(f"   Precision: {precision:.2f} ({rel_count}/3 문서가 관련 있음)")
print(f"   Recall: {recall:.2f} ({found_count}/{len(required_recipes)} 필요 레시피 찾음)")
print()

print("📝 검색된 레시피:")
for i, doc in enumerate(results, 1):
    recipe_name = doc.metadata.get('recipe_name', 'Unknown')
    print(f"   {i}. {recipe_name}")
print()

if threshold > 0.5:
    print("⚠️ threshold 높음: Precision이 낮을 수 있습니다")
elif precision >= 0.6 and recall >= 0.8:
    print("✅ 균형잡힌 설정!")

In [ ]:
# 테스트 질문 (애매한 질문)
test_query = "아침 메뉴로 좋은 거"
required_recipes = ["김치볶음밥"]

# 검색
results = vectorstore.similarity_search(test_query, k=3)

# 평가
precision, rel_count = calculate_context_precision(test_query, results, embeddings, threshold=threshold)
recall, found_count = calculate_context_recall(required_recipes, results)

# 출력
print(f"🔍 질문: '{test_query}'")
print(f"   정답: {required_recipes}")
print()
print(f"📊 결과 (threshold={threshold}):")
print(f"   Precision: {precision:.2f} ({rel_count}/3 문서가 관련 있음)")
print(f"   Recall: {recall:.2f} ({found_count}/{len(required_recipes)} 필요 레시피 찾음)")
print()

if recall == 0:
    print("⚠️ 애매한 질문: threshold↓ 또는 k↑ 필요")
elif recall >= 1.0:
    print("✅ 애매한 질문도 성공!")

In [ ]:
# ============================================
# 여기를 수정하세요! 👇
# ============================================
my_chunk_size = 100  # ← 300~500 추천!
my_top_k = 1         # ← 3~5 추천!
my_threshold = 0.7   # ← 0.3~0.5 추천!

# ⚠️ 현재 설정은 의도적으로 나쁩니다!
# 위 3개 파라미터를 모두 바꿔서 최고 성능을 만들어보세요!
# ============================================

# 벡터스토어 재생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=my_chunk_size,
    chunk_overlap=20,
    length_function=len
)

documents = []
for doc_data in recipe_docs_raw:
    chunks = text_splitter.split_text(doc_data["content"])
    for chunk in chunks:
        documents.append(Document(
            page_content=chunk,
            metadata={"recipe_name": doc_data["recipe_name"]}
        ))

vectorstore = FAISS.from_documents(documents, embeddings)

print("🎯 나만의 최적 설정 테스트")
print("=" * 60)
print(f"chunk_size={my_chunk_size}, top_k={my_top_k}, threshold={my_threshold}")
print()

# 5개 질문 테스트
test_cases = [
    {"query": "찌개 종류 중에 두부 들어가고 매운 거 뭐 있어?", "required": ["김치찌개", "된장찌개"]},
    {"query": "고기 들어가는 매운 요리 추천해줘", "required": ["김치찌개"]},
    {"query": "고기 들어가는 매콤한 볶음요리", "required": ["제육볶음", "백종원 제육볶음"]},
    {"query": "간단한 밥 요리", "required": ["김치볶음밥"]},
    {"query": "한 그릇 식사", "required": ["비빔밥"]},
]

total_precision = 0
total_recall = 0

for i, test in enumerate(test_cases, 1):
    results = vectorstore.similarity_search(test["query"], k=my_top_k)
    precision, rel_count = calculate_context_precision(test["query"], results, embeddings, threshold=my_threshold)
    recall, found_count = calculate_context_recall(test["required"], results)
    
    total_precision += precision
    total_recall += recall
    
    print(f"[{i}] {test['query'][:30]}...")
    print(f"    Precision: {precision:.2f}, Recall: {recall:.2f}")

avg_precision = total_precision / len(test_cases)
avg_recall = total_recall / len(test_cases)

print()
print("=" * 60)
print(f"📊 평균 Precision: {avg_precision:.2f}")
print(f"📊 평균 Recall: {avg_recall:.2f}")
print()

if avg_precision >= 0.6 and avg_recall >= 0.8:
    print("🎉 성공! 최적 설정을 찾았습니다!")
elif avg_recall < 0.5:
    print("⚠️ Recall 낮음: chunk_size↑ 또는 top_k↑ 추천")
elif avg_precision < 0.4:
    print("⚠️ Precision 낮음: threshold↑ 또는 top_k↓ 추천")
else:
    print("💡 조금만 더 조정하면 완벽!")

In [ ]:
# ============================================
# 여러분의 최적 설정을 찾아보세요! 👇
# ============================================

# ⚠️ 경고: 아래 설정은 의도적으로 극단적입니다!
# 단순한 질문에서는 성공할 수 있지만, 복잡한 질문에서 실패합니다!

# 단순 질문용 설정
config_simple = {
    "chunk_size": 50,
    "top_k": 1,
    "threshold": 0.8
}

# 중간 질문용 설정
config_medium = {
    "chunk_size": 50,
    "top_k": 2,
    "threshold": 0.75
}

# 복합 질문용 설정
config_complex = {
    "chunk_size": 50,
    "top_k": 1,
    "threshold": 0.9
}

# 테스트 케이스 (완전 간접적 질문!)
test_cases = [
    {
        "type": "단순",
        "query": "매운 국물 요리 추천해줘",
        "required": ["김치찌개"],
        "config": config_simple
    },
    {
        "type": "중간",
        "query": "백종원 레시피 뭐있어?",
        "required": ["백종원 참치김치찌개", "백종원 제육볶음"],
        "config": config_medium
    },
    {
        "type": "복합",
        "query": "요리 추천해줘",
        "required": ["김치찌개", "제육볶음", "불고기"],
        "config": config_complex
    }
]

print("🎯 간접 질문 테스트")
print("=" * 80)

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
import pandas as pd

results_summary = []

for test in test_cases:
    config = test["config"]
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=config["chunk_size"],
        chunk_overlap=20
    )
    
    temp_docs = []
    for doc_data in recipe_docs_raw:
        chunks = text_splitter.split_text(doc_data["content"])
        for chunk in chunks:
            temp_docs.append(Document(
                page_content=chunk,
                metadata={"recipe_name": doc_data["recipe_name"]}
            ))
    
    temp_vectorstore = FAISS.from_documents(temp_docs, embeddings)
    search_results = temp_vectorstore.similarity_search(test["query"], k=config["top_k"])
    
    precision, _ = calculate_context_precision(test["query"], search_results, embeddings, threshold=config["threshold"])
    recall, _ = calculate_context_recall(test["required"], search_results)
    
    success = "✅" if (precision >= 0.6 and recall >= 0.8) else "⚠️"
    
    print(f"\n[{test['type']}] \"{test['query']}\"")
    print(f"   Precision={precision:.2f}, Recall={recall:.2f} {success}")
    
    results_summary.append({
        "질문 타입": test["type"],
        "Precision": f"{precision:.2f}",
        "Recall": f"{recall:.2f}",
        "통과": success
    })

print("\n" + "=" * 80)
df = pd.DataFrame(results_summary)
print(df.to_string(index=False))

success_count = sum(1 for r in results_summary if r["통과"] == "✅")
total_count = len(results_summary)
print(f"\n🎯 성공률: {success_count}/{total_count} ({success_count/total_count*100:.0f}%)")

if success_count == 0:
    print("\n⚠️ 모든 테스트 실패!")
    print("   → 위에서 config 값들을 수정하고 다시 실행하세요!")
    print()
    print("💡 권장 개선 방향:")
    print("   1. chunk_size: 50 → 300~500")
    print("   2. top_k: 1~2 → 3~5")
    print("   3. threshold: 0.75~0.9 → 0.3~0.5")
elif success_count < total_count:
    print("\n⚠️ 일부 테스트 실패!")
    print("   → 실패한 케이스의 파라미터를 조정해보세요!")
else:
    print("\n🎉 모든 테스트 성공! 축하합니다!")
    print("   → 여러분만의 최적 파라미터를 찾았습니다!")

## 📝 Part 2: 나만의 평가 데이터셋 만들기

### 핵심 아이디어

**"내가 관심있는 도메인으로 RAG를 테스트해보자!"**

### 데이터셋 만들기 3단계

1. **도메인 선택**: 레시피, 영화, IT, 게임 공략 등
2. **테스트 케이스 작성**: 질문 10개 + 정답 작성
3. **자동 평가**: 07번 RAGAS로 평가

### 템플릿

아래 템플릿을 참고해서 직접 만들어보세요!

In [ ]:
# ============================================
# 여러분만의 테스트 데이터셋을 만들어보세요! 👇
# ============================================

my_custom_dataset = [
    # 예시 1: 간단한 질문
    {
        "query": "매운 국물 요리",
        "required_recipes": ["김치찌개"]
    },
    
    # 예시 2: 구체적 질문
    {
        "query": "돼지고기 볶음 요리",
        "required_recipes": ["제육볶음"]
    },
    
    # 예시 3: 복합 질문
    {
        "query": "찌개 종류 추천해줘",
        "required_recipes": ["김치찌개", "된장찌개"]
    },
    
    # ============================================
    # 아래에 여러분만의 케이스를 추가하세요!
    # ============================================
    
    # 예시 4: 여러분의 케이스
    # {
    #     "query": "여러분의 질문",
    #     "required_recipes": ["정답 레시피1", "정답 레시피2"]
    # },
]

print(f"✅ 커스텀 데이터셋 준비 완료!")
print(f"   총 {len(my_custom_dataset)}개 테스트 케이스")
print()
print("테스트 케이스:")
for i, case in enumerate(my_custom_dataset, 1):
    print(f"   {i}. {case['query']}")
    print(f"      → 정답: {case['required_recipes']}")

In [ ]:
# 커스텀 데이터셋 평가
print("🔬 커스텀 데이터셋 평가 시작")
print("=" * 80)

# 기본 설정으로 벡터스토어 재생성
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
eval_docs = []

for doc_data in recipe_docs_raw:
    chunks = text_splitter.split_text(doc_data["content"])
    for chunk in chunks:
        eval_docs.append(Document(
            page_content=chunk,
            metadata={"recipe_name": doc_data["recipe_name"]}
        ))

eval_vectorstore = FAISS.from_documents(eval_docs, embeddings)

# 커스텀 데이터셋
my_custom_dataset = [
    {"query": "김치 들어가는 음식 뭐있어?", "required": ["김치찌개", "백종원 참치김치찌개", "김치볶음밥"]},
    {"query": "아침에 빨리 만들 수 있는 거 추천", "required": ["김치볶음밥"]},
    {"query": "찌개 종류 알려줘", "required": ["김치찌개", "된장찌개", "순두부찌개"]},
]

print(f"📝 평가 데이터셋: {len(my_custom_dataset)}개 질문")
print()

custom_results = []
for i, test in enumerate(my_custom_dataset, 1):
    query = test["query"]
    required = test["required"]
    
    # 검색
    results = eval_vectorstore.similarity_search(query, k=5)
    
    # 평가
    precision, rel_count = calculate_context_precision(query, results, embeddings, threshold=0.3)
    recall, found_count = calculate_context_recall(required, results)
    
    print(f"[질문 {i}] \"{query}\"")
    print(f"   정답: {len(required)}개 레시피")
    print(f"   Precision: {precision:.2f}, Recall: {recall:.2f}")
    print()
    
    custom_results.append({
        "질문": query,
        "Precision": precision,
        "Recall": recall
    })

# 평균 계산
avg_precision = sum(r["Precision"] for r in custom_results) / len(custom_results)
avg_recall = sum(r["Recall"] for r in custom_results) / len(custom_results)

print("=" * 80)
print(f"📊 평균 성능: Precision={avg_precision:.2f}, Recall={avg_recall:.2f}")

## 🚀 Part 3: Gradio UI로 빠른 프로토타입

### 핵심 아이디어

**"3줄 코드로 RAG 챗봇 UI 만들기!"**

### Gradio란?

- Python 함수를 **웹 UI로 변환**해주는 라이브러리
- Colab에서 바로 실행 가능 (Public URL 생성)
- **⚠️ Kaggle에서는 작동하지 않습니다**

### 환경별 실행 방법

| 환경 | 실행 방법 | 비고 |
|------|-----------|------|
| **Colab** | `share=True` → Public URL 생성 | ✅ 정상 작동 |
| **Kaggle** | ❌ Gradio 미지원 | 함수 직접 호출 사용 |
| **로컬** | `localhost:7860` | ✅ 정상 작동 |

### Kaggle 사용자를 위한 대안

Gradio 대신 함수를 직접 호출하세요:

```python
# Gradio 대신 이렇게 사용
result = rag_chatbot(
    query="매운 국물 요리 추천해줘", 
    strategy="Naive RAG", 
    top_k=3
)
print(result)
```

### 실습: RAG 챗봇 UI 만들기

아래 코드를 실행하면:
- **Colab**: 챗봇 UI + Public URL
- **Kaggle**: 함수 직접 호출 안내

In [ ]:
# Gradio 및 LLM 라이브러리 설치
!pip install -q gradio transformers accelerate

# EXAONE 모델 로드
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("🔧 EXAONE 모델 로딩 중...")
model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print("✅ EXAONE 모델 로드 완료!")

In [ ]:
import gradio as gr

# RAG 답변 생성 함수
def generate_answer(query, context):
    """
    EXAONE으로 RAG 답변 생성
    
    Args:
        query: 사용자 질문
        context: 검색된 문서 내용
    """
    prompt = f"""당신은 한식 레시피 전문가입니다. 아래 검색된 레시피 정보를 바탕으로 사용자의 질문에 답변하세요.

[검색된 레시피]
{context}

[질문]
{query}

[답변]
"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # prompt 제거하고 답변만 추출
    if "[답변]" in answer:
        answer = answer.split("[답변]")[-1].strip()
    
    return answer

# RAG 챗봇 함수
def rag_chatbot(query, top_k, chunk_size):
    """
    실제 RAG 챗봇 (검색 + LLM 답변)
    
    Args:
        query: 사용자 질문
        top_k: 검색 결과 개수
        chunk_size: Chunk 크기
    """
    try:
        # 1. 벡터스토어 생성 (동적)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=int(chunk_size),
            chunk_overlap=20,
            length_function=len
        )
        
        documents = []
        for doc_data in recipe_docs_raw:
            chunks = text_splitter.split_text(doc_data["content"])
            for chunk in chunks:
                documents.append(Document(
                    page_content=chunk,
                    metadata={"recipe_name": doc_data["recipe_name"]}
                ))
        
        temp_vectorstore = FAISS.from_documents(documents, embeddings)
        
        # 2. 검색
        results = temp_vectorstore.similarity_search(query, k=int(top_k))
        
        if not results:
            return "검색 결과가 없습니다."
        
        # 3. Context 생성
        context = ""
        for doc in results:
            recipe_name = doc.metadata.get('recipe_name', 'Unknown')
            context += f"[{recipe_name}]\n{doc.page_content}\n\n"
        
        # 4. LLM 답변 생성
        answer = generate_answer(query, context)
        
        # 5. 출력 포맷
        output = f"**💬 답변:**\n\n{answer}\n\n"
        output += "---\n\n"
        output += f"**📚 참고한 레시피 ({len(results)}개):**\n"
        for i, doc in enumerate(results, 1):
            recipe = doc.metadata.get('recipe_name', 'Unknown')
            output += f"{i}. {recipe}\n"
        
        return output
        
    except Exception as e:
        return f"오류 발생: {str(e)}"

# Gradio UI 생성
interface = gr.Interface(
    fn=rag_chatbot,
    inputs=[
        gr.Textbox(
            label="질문",
            placeholder="예: 김치찌개 만드는 법 알려줘",
            lines=2
        ),
        gr.Slider(
            minimum=1,
            maximum=5,
            value=3,
            step=1,
            label="Top-K (검색 개수)"
        ),
        gr.Slider(
            minimum=100,
            maximum=1000,
            value=400,
            step=100,
            label="Chunk Size"
        )
    ],
    outputs=gr.Textbox(
        label="답변",
        lines=15
    ),
    title="🍳 한식 레시피 RAG 챗봇",
    description="질문하면 EXAONE이 레시피 정보를 검색해서 답변해드립니다!",
    examples=[
        ["김치찌개 만드는 법 알려줘", 3, 400],
        ["고기 들어가는 매운 요리 추천해줘", 3, 400],
        ["간단한 아침 메뉴 추천", 5, 400],
    ]
)

# 실행
# Colab: share=True로 Public URL 생성
# Kaggle: 작동 안 함 (함수 직접 호출 사용)
interface.launch(share=True)

### 💡 Gradio UI 활용 팁

**1. Colab에서 Public URL 생성하기**

```python
interface.launch(share=True)  # ← 이렇게만 하면 끝!
```

실행하면:
```
Running on public URL: https://abc123.gradio.live
```

이 링크를 복사해서 누구에게나 보낼 수 있어요! (72시간 유효)

**2. ⚠️ Kaggle에서는 Gradio가 작동하지 않습니다**

Kaggle 환경에서는 함수를 직접 호출하세요:

```python
# Gradio 대신 이렇게 사용
result = rag_chatbot(
    query="매운 국물 요리 추천해줘", 
    strategy="Naive RAG", 
    top_k=3
)
print(result)
```

**3. 더 많은 기능 추가하기**

- 이미지 업로드: `gr.Image()`
- 파일 업로드: `gr.File()`
- 오디오: `gr.Audio()`
- 채팅 UI: `gr.ChatInterface()`

### 리소스

- [Gradio 공식 문서](https://gradio.app)
- [Gradio 예제 모음](https://gradio.app/demos)

## 🎓 Part 4: 최종 정리 & Day 3 Preview

### 오늘 배운 것 체크리스트

**Part 1: 파라미터 튜닝**
- [ ] Chunk Size가 검색에 미치는 영향 이해
- [ ] Top-K Precision/Recall 트레이드오프 체험
- [ ] Threshold 조정 효과 확인
- [ ] 질문 타입별 최적 파라미터 조합 발견

**Part 2: 커스텀 데이터셋**
- [ ] 나만의 평가 데이터셋 작성
- [ ] 자동 평가 실행
- [ ] 결과 분석 및 개선점 파악

**Part 3: Gradio UI**
- [ ] 3줄 코드로 UI 만들기
- [ ] Colab에서 실행 및 테스트
- [ ] (선택) Public URL 생성 및 공유

---

### Day 2 완전한 여정 (01-08번)

```
01번: Naive RAG 베이스라인
  → 복합 질문 실패 ❌

02번: 실패 원인 분석
  → 5가지 케이스 진단 🔍

03번: Multi-Query Retrieval
  → 질문 분해로 해결 ✅

04번: Metadata Filtering
  → 동음이의어 해결 ✅

05번: Hybrid + Reranking
  → 검색 정확도 극대화 ✅

06번: Corrective RAG (CRAG)
  → 검색 검증 + 폴백 ✅

07번: RAG Evaluation (RAGAS)
  → 정량적 평가로 최선 선택 ✅

08번: Tuning & Customization ← 여기!
  → 직접 만지고, 실험하고, 나만의 것 만들기 ✅✅✅
```

---

### 🚀 Day 3 Preview: Agent + LangGraph

**Day 2에서 배운 것:**
- "어떻게" RAG를 만드는가? (기법들)
- "어떻게" 평가하는가? (RAGAS)
- "어떻게" 튜닝하는가? (파라미터)

**Day 3에서 배울 것:**
- "언제" 어떤 도구를 쓸까? (Agent 결정)
- "어떻게" 복잡한 워크플로우를 만들까? (LangGraph)
- "어떻게" 여러 Agent를 협업시킬까? (Multi-Agent)

**예시: Day 3 Agent**

```
질문: "김치찌개 만드는 법 알려줘, 그리고 백종원 버전도 궁금해"

Agent 사고 과정:
1. 질문 분석 → "레시피 + 변형 버전" 필요
2. 도구 선택:
   - RAG 검색 (정통 레시피)
   - RAG 검색 (백종원 버전)
3. 결과 통합 → 최종 답변

→ LangGraph로 이 흐름을 자동화!
```

---

### 💡 실무 활용 가이드

**새 도메인에 RAG 적용할 때:**
1. **01-02번**: Naive RAG로 베이스라인 구축
2. **08번**: 파라미터 튜닝 실험 (chunk, k, threshold)
3. **07번**: 커스텀 데이터셋으로 평가
4. **03-06번**: 성능 낮으면 고급 기법 적용
5. **08번**: Gradio로 프로토타입 → 이해관계자 데모

**파라미터 튜닝 가이드:**

- Chunk Size:
  - 단순 질문: 200-300자
  - 복합 질문: 500-700자
  - 긴 문맥: 1000자+

- Top-K:
  - 빠른 응답: k=1-3
  - 정확도 우선: k=5-7
  - 포괄 검색: k=10+

- Threshold:
  - 엄격한 도메인: 0.5-0.7
  - 일반 도메인: 0.3-0.5
  - 탐색적 검색: 0.2-0.3

---

## 🎉 Day 2 완전 정복!

**여러분은 이제:**
- ✅ Advanced RAG 모든 기법 마스터
- ✅ 파라미터 튜닝 능력 보유
- ✅ 평가 및 최적화 가능
- ✅ 커스텀 RAG 시스템 구축 가능
- ✅ 프로토타입 UI까지 완성!

**다음 단계:**
- Day 3에서 LangGraph Agent로 만나요! 🚀

**질문이나 피드백:**
- GitHub Issues에 남겨주세요!
- 실무에서 써보고 경험 공유해주세요!